In [ ]:
import csv
import random

class TabelaHash:
    # Aceita uma função de hash no construtor
    def __init__(self, tamanho=100, funcao_hash=None):
        self.tamanho = tamanho
        self.tabela = [[] for _ in range(tamanho)]
        
        # Se nenhuma função for passada, usa uma padrão
        if funcao_hash is None:
            self.funcao_hash = self.hash_padrao
        else:
            self.funcao_hash = funcao_hash

    def hash_padrao(self, chave):
        # Função de hash baseada na função hash nativa do Python
        return hash(chave) % self.tamanho

    def inserir(self, chave, dado):
        indice = self.funcao_hash(chave)
        # Verifica se a chave já existe (duplicata)
        for item in self.tabela[indice]:
            if item[0] == chave:
                return False  # Duplicata, não insere
        self.tabela[indice].append((chave, dado))
        return True

    def buscar(self, chave):
        indice = self.funcao_hash(chave)
        for item in self.tabela[indice]:
            if item[0] == chave:
                return item[1]
        return None

    def remover(self, chave):
        indice = self.funcao_hash(chave)
        self.tabela[indice] = [item for item in self.tabela[indice] if item[0] != chave]

    def elementos(self):
        for lista in self.tabela:
            for chave, dado in lista:
                yield dado

def gerar_csv_aleatorio(nome_arquivo, num_registros=30):
    nomes = ['Ana', 'Bruno', 'Carla', 'Daniel', 'Eduarda', 'Felipe', 'Gabriela', 'Henrique']
    anos = [2020, 2021, 2022, 2023, 2024]
    registros = []

    for i in range(num_registros):
        nome = random.choice(nomes)
        ano = random.choice(anos)
        # Cria um id único para cada linha, mas permite duplicatas em nome/ano
        registro = {
            'id': str(i + 1),
            'nome': nome,
            'ano': str(ano)
        }
        registros.append(registro)

    # Adiciona algumas duplicatas propositalmente
    for _ in range(num_registros // 5):
        registro_dup = random.choice(registros)
        registros.append(registro_dup.copy())

    # Escreve no arquivo CSV
    with open(nome_arquivo, 'w', newline='', encoding='utf-8') as csvfile:
        campos = ['id', 'nome', 'ano']
        escritor = csv.DictWriter(csvfile, fieldnames=campos)
        escritor.writeheader()
        for reg in registros:
            # print(registros)
            escritor.writerow(reg)

    print(f'Arquivo CSV "{nome_arquivo}" gerado com {len(registros)} registros (com duplicatas).')

# Função principal para ler o CSV e eliminar duplicatas
def eliminar_duplicatas_csv(nome_arquivo_csv, campos_chave):
    tabela = TabelaHash(tamanho=1000)
    with open(nome_arquivo_csv, newline='', encoding='utf-8') as csvfile:
        leitor = csv.DictReader(csvfile)
        for linha in leitor:
            # Monta a chave como tupla dos campos escolhidos para identificar duplicatas
            chave = tuple(linha[campo] for campo in campos_chave)
            tabela.inserir(chave, linha)
    # Retorna os elementos únicos
    return list(tabela.elementos())

# Exemplo de uso
if __name__ == "__main__":
    # Supondo que seu CSV tem colunas: id, nome, ano
    nome_arquivo = "dados.csv"
    campos_chave = ['nome', 'ano']  # Defina os campos que identificam duplicatas

    # Gera um CSV aleatório para teste
    gerar_csv_aleatorio(nome_arquivo, num_registros=30)

    dados_unicos = eliminar_duplicatas_csv(nome_arquivo, campos_chave)

    print("Registros únicos encontrados:")
    for registro in dados_unicos:
        print(registro)